# Sakila: join entre tablas

In [1]:
# mostrar un dataframe de empleados con su dirección completa
# staff, address, city, country
import pandas as pd
import mysql.connector as connector

## Carga de datos

In [2]:
connection = connector.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="sakila"
)

In [3]:
df_tables = pd.read_sql("SHOW TABLES;", connection)
df_tables.head(20)

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\2844210054.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql("SHOW TABLES;", connection)


,Tables_in_sakila
0,actor
1,actor_info
2,address
3,category
4,city
5,country
6,customer
7,customer_list
8,film
9,film_actor


In [4]:
df_staff = pd.read_sql("SELECT * FROM staff", connection)
df_address = pd.read_sql("SELECT * FROM address", connection)
df_city = pd.read_sql("SELECT * FROM city", connection)
df_country = pd.read_sql("SELECT * FROM country", connection)

df_staff.head()

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\732414893.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_staff = pd.read_sql("SELECT * FROM staff", connection)
C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\732414893.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_address = pd.read_sql("SELECT * FROM address", connection)
C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\732414893.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_city = pd.read_sql("SELECT * FROM city", connection)
C:\Users\alanj\AppDa

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-02-15 03:57:16
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,None,2006-02-15 03:57:16


In [5]:
df_address.head(1)

,address_id,address,address2,district,city_id,postal_code,phone,location,last_update
0,1,47 MySakila Drive,None,Alberta,300,,,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 62, 10, 50, 93, 99...",2014-09-25 22:30:27


## Address de Empleados

In [6]:
df_staff_address = pd.merge(df_staff, df_address, on='address_id')
df_staff_address = df_staff_address[['first_name', 'last_name', 'address', 'city_id']]

df_staff_address_city = pd.merge(df_staff_address, df_city, on='city_id')
df_staff_address_city = df_staff_address_city[
    ['first_name', 'last_name', 'address', 'city_id', 'city', 'country_id']
]

df_staff_direction = pd.merge(df_staff_address_city, df_country, on='country_id')
df_staff_direction = df_staff_direction[
    ['first_name', 'last_name', 'address', 'city', 'country']
]

df_staff_direction.head()

,first_name,last_name,address,city,country
0,Mike,Hillyer,23 Workhaven Lane,Lethbridge,Canada
1,Jon,Stephens,1411 Lillydale Drive,Woodridge,Australia


In [7]:
df_staff_address = pd.merge(df_staff, df_address, on='address_id')[['first_name', 'last_name', 'address', 'city_id']]
df_staff_address_city = pd.merge(df_staff_address, df_city, on='city_id')[['first_name', 'last_name', 'address', 'city_id', 'city', 'country_id']]
df_staff_direction = pd.merge(df_staff_address_city, df_country, on='country_id')[['first_name', 'last_name', 'address', 'city', 'country']]
df_staff_direction.head()

,first_name,last_name,address,city,country
0,Mike,Hillyer,23 Workhaven Lane,Lethbridge,Canada
1,Jon,Stephens,1411 Lillydale Drive,Woodridge,Australia


## Concatenar columnas string

In [8]:
# concatenar columna first_name y last_name
df_staff_direction['full_name'] = df_staff_direction['first_name'].str.cat(
                                    df_staff_direction['last_name'], 
                                    sep=" "
                                    )

df_staff_direction

,first_name,last_name,address,city,country,full_name
0,Mike,Hillyer,23 Workhaven Lane,Lethbridge,Canada,Mike Hillyer
1,Jon,Stephens,1411 Lillydale Drive,Woodridge,Australia,Jon Stephens


## Extracción de texto de un String

In [14]:
# extraer las iniciales de firstname y lastname y guardarlas en una nueva columna
df_staff_direction['short_name'] = df_staff_direction['first_name'].str[0] + df_staff_direction['last_name'].str[0]
df_staff_direction.head()

,first_name,last_name,address,city,country,full_name,short_name
0,Mike,Hillyer,23 Workhaven Lane,Lethbridge,Canada,Mike Hillyer,MH
1,Jon,Stephens,1411 Lillydale Drive,Woodridge,Australia,Jon Stephens,JS


## Agrupar pagos por empleado

In [17]:
# Mostrar suma total de pagos registrados por cada staff / empleado
df_payment = pd.read_sql("SELECT * FROM payment", connection)
df_payment.head(2)

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\1216999880.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_payment = pd.read_sql("SELECT * FROM payment", connection)


,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30


In [18]:
df_payment.groupby('staff_id')['amount'].sum()

staff_id
1    33482.50
2    33924.06
Name: amount, dtype: float64

In [23]:
# crear nuevo dataframe que tenga el nombre de cada empleado y
# el amount (cantidad dinero facturada) total de cada uno

df_staff_payment = pd.DataFrame()

df_staff = pd.read_sql('SELECT * FROM staff;', connection, index_col='staff_id')
df_payment = pd.read_sql("SELECT * FROM payment;", connection)

# guardar columnas de dataframes distintos en un dataframe nuevo df_staff_payment
df_staff_payment['full_name'] = df_staff['first_name'] + ' ' + df_staff['last_name'] # df_staff
df_staff_payment['payment'] = df_payment.groupby('staff_id')['amount'].sum() # df_payment
df_staff_payment

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\926257572.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_staff = pd.read_sql('SELECT * FROM staff;', connection, index_col='staff_id')
C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\926257572.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_payment = pd.read_sql("SELECT * FROM payment", connection)


,full_name,payment
staff_id,,
1,Mike Hillyer,33482.50
2,Jon Stephens,33924.06


In [24]:
df_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   payment_id    16044 non-null  int64         
 1   customer_id   16044 non-null  int64         
 2   staff_id      16044 non-null  int64         
 3   rental_id     16044 non-null  int64         
 4   amount        16044 non-null  float64       
 5   payment_date  16044 non-null  datetime64[ns]
 6   last_update   16044 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(4)
memory usage: 877.5 KB


## Agrupar pagos por fechas

In [26]:
# Extraer el mes de la columna payment_date
df_payment['month'] = pd.to_datetime(df_payment['payment_date']).dt.month
df_payment['year'] = pd.to_datetime(df_payment['payment_date']).dt.year
df_payment.head(2)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update,month,year
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30,5,2005
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30,5,2005


In [29]:
df_payment.describe()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update,month,year
count,16044.000000,16044.000000,16044.000000,16044.000000,16044.000000,16044,16044,16044.000000,16044.000000
mean,8024.483732,297.143169,1.498005,8025.371478,4.201356,2005-07-23 08:12:53.217526912,2006-02-15 22:16:22.966404864,7.009536,2005.011344
min,1.000000,1.000000,1.000000,1.000000,0.000000,2005-05-24 22:53:30,2006-02-15 22:12:30,2.000000,2005.000000
25%,4012.750000,148.000000,1.000000,4013.750000,2.990000,2005-07-07 00:58:40.500000,2006-02-15 22:13:28,7.000000,2005.000000
50%,8024.500000,296.000000,1.000000,8025.500000,3.990000,2005-07-28 16:04:32.500000,2006-02-15 22:15:37,7.000000,2005.000000
75%,12036.250000,446.000000,2.000000,12037.250000,4.990000,2005-08-17 21:16:23,2006-02-15 22:18:57,8.000000,2005.000000
max,16049.000000,599.000000,2.000000,16049.000000,11.990000,2006-02-14 15:16:03,2006-02-15 22:24:13,8.000000,2006.000000
std,4632.695501,172.453136,0.500012,4632.777249,2.362961,NaN,NaN,1.034514,0.105905


In [30]:
df_payment['month'].value_counts()

month
7    6709
8    5686
6    2311
5    1156
2     182
Name: count, dtype: int64

In [31]:
df_payment['year'].value_counts()

year
2005    15862
2006      182
Name: count, dtype: int64

In [32]:
# agrupar pagos por mes
pd.DataFrame(df_payment.groupby('month')['amount'].sum())

,amount
month,
2,514.18
5,4823.44
6,9629.89
7,28368.91
8,24070.14


In [33]:
# agrupar pagos por año
pd.DataFrame(df_payment.groupby('year')['amount'].sum())

,amount
year,
2005,66892.38
2006,514.18


In [35]:
# agrupar pagos por año y mes
pd.DataFrame(df_payment.groupby(['year', 'month'])['amount'].sum())

amount
year month          
2005 5       4823.44
     6       9629.89
     7      28368.91
     8      24070.14
2006 2        514.18

In [36]:
# agrupar pagos por empleado, año y mes
pd.DataFrame(df_payment.groupby(['staff_id', 'year', 'month'])['amount'].sum())

amount
staff_id year month          
1        2005 5       2621.83
              6       4774.37
              7      13998.56
              8      11853.65
         2006 2        234.09
2        2005 5       2201.61
              6       4855.52
              7      14370.35
              8      12216.49
         2006 2        280.09

## Count de actores por película

Cargar la tabla film_actor

In [37]:
df_film_actor = pd.read_sql('SELECT * FROM film_actor;', connection)
df_film_actor.head(2)

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\3763140905.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_film_actor = pd.read_sql('SELECT * FROM film_actor;', connection)


,actor_id,film_id,last_update
0,1,1,2006-02-15 05:05:03
1,1,23,2006-02-15 05:05:03


In [38]:
df_film_actor.groupby('film_id')['actor_id'].count()

film_id
1       10
2        4
3        5
4        5
5        5
        ..
996      5
997      5
998      6
999      5
1000     3
Name: actor_id, Length: 997, dtype: int64

In [39]:
# Alternativa: si no tuvieramos la función count() se podría utilizar sum()
df_film_actor['count'] = 1
df_film_actor.head(5)

,actor_id,film_id,last_update,count
0,1,1,2006-02-15 05:05:03,1
1,1,23,2006-02-15 05:05:03,1
2,1,25,2006-02-15 05:05:03,1
3,1,106,2006-02-15 05:05:03,1
4,1,140,2006-02-15 05:05:03,1


In [41]:
serie = df_film_actor.groupby('film_id')['count'].sum()
df_film_actor_count = pd.DataFrame(serie)
df_film_actor_count.sort_values('count', ascending=False, inplace=True)
df_film_actor_count

,count
film_id,
508,15
188,13
714,13
87,13
146,13
...,...
789,1
611,1
328,1


In [43]:
df_film = pd.read_sql('SELECT * FROM film;', connection, index_col='film_id')
df_film_actor_count['title'] = df_film['title'] # índice común permite joins automáticos
df_film_actor_count.head(3)

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\1882689578.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_film = pd.read_sql('SELECT * FROM film;', connection, index_col='film_id')


,count,title
film_id,,
508,15,LAMBS CINCINATTI
188,13,CRAZY HOME
714,13,RANDOM GO


In [45]:
# cambiar los nombres de las columnas usando fancy indexing
df_film_actor_count.columns = [['num_actors', 'film_title']]
df_film_actor_count.head(3)


,num_actors,film_title
film_id,,
508,15,LAMBS CINCINATTI
188,13,CRAZY HOME
714,13,RANDOM GO


## ¿Cuántas copias existen de la película CRAZY HOME en stock?

Usar la tabla stock/inventario inventory

In [47]:
df_inventory = pd.read_sql('SELECT * FROM inventory;', connection)
df_inventory.head(3)


C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\2637493878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_inventory = pd.read_sql('SELECT * FROM inventory;', connection)


,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17


In [60]:
# filtrar en df_film la película CRAZY HOME y quedarnos con su id
filter = df_film['title'] == 'ACADEMY DINOSAUR'
film_crazy_home = df_film[filter]
film_crazy_home

,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
film_id,,,,,,,,,,,,
1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42


In [61]:
film_crazy_home.index

Index([1], dtype='int64', name='film_id')

In [62]:
film_id = film_crazy_home.index[0]
film_id

1

In [63]:
filter = df_inventory['film_id'] == film_id
df_inventory_crazy_home = df_inventory[filter]
df_inventory_crazy_home['film_id'].count()

8

In [64]:
# count pero agrupando por tienda, probar con ACADEMY DINOSAUR
df_inventory_crazy_home.groupby('store_id')['film_id'].count()

store_id
1    4
2    4
Name: film_id, dtype: int64

## ¿Cuánto ha pagado en total cada customer? De más a menos

Mostrando firstname, lastname, email del cliente

In [68]:
df_customer = pd.read_sql('SELECT * FROM customer;', connection, index_col='customer_id')
df_customer['amount'] = df_payment.groupby('customer_id')['amount'].sum()
df_customer.head(3)

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\3347528981.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer = pd.read_sql('SELECT * FROM customer;', connection, index_col='customer_id')


,store_id,first_name,last_name,email,address_id,active,create_date,last_update,amount
customer_id,,,,,,,,,
1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,118.68
2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20,128.73
3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20,135.74


## Mostrar las películas más alquiladas

rental

In [69]:
# merge de rental con inventory
df_rental = pd.read_sql('SELECT * FROM rental;', connection)
df_inventory = pd.read_sql('SELECT * FROM inventory;', connection)

df_rental_inventory = pd.merge(df_rental, df_inventory, on='inventory_id')
df_rental_inventory.head(3)

C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\3985288158.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rental = pd.read_sql('SELECT * FROM rental;', connection)
C:\Users\alanj\AppData\Local\Temp\ipykernel_7252\3985288158.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_inventory = pd.read_sql('SELECT * FROM inventory;', connection)


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update_x,film_id,store_id,last_update_y
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,80,1,2006-02-15 05:09:17
1,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,2006-02-15 21:30:53,80,1,2006-02-15 05:09:17
2,3584,2005-07-06 04:16:43,367,207,2005-07-13 07:08:43,1,2006-02-15 21:30:53,80,1,2006-02-15 05:09:17


In [85]:
rental_counts = df_rental_inventory['film_id'].value_counts()
df_rental_counts = pd.DataFrame(rental_counts)
df_rental_counts.head(3)

,count
film_id,
103,34
738,33
331,32


In [93]:
df_film['times_rented'] = df_rental_counts['count']
df_film.sort_values('times_rented', ascending=False, inplace=True)
df_film.isnull().sum()

title                      0
description                0
release_year               0
language_id                0
original_language_id    1000
rental_duration            0
rental_rate                0
length                     0
replacement_cost           0
rating                     0
special_features           0
last_update                0
times_rented              42
dtype: int64

In [73]:
# Ejemplo equivalente al value_counts()
df_rental_inventory.groupby('film_id')['film_id'].count()

film_id
1       23
2        7
3       12
4       23
5       12
        ..
996      7
997      6
998      9
999     17
1000    31
Name: film_id, Length: 958, dtype: int64

In [94]:
# rellenar valores nulos con un 0 para indicar que la film ha sido 0 veces alquilada
df_film['times_rented'].fillna(0, inplace=True)
df_film.isnull().sum()

title                      0
description                0
release_year               0
language_id                0
original_language_id    1000
rental_duration            0
rental_rate                0
length                     0
replacement_cost           0
rating                     0
special_features           0
last_update                0
times_rented               0
dtype: int64

In [95]:
df_film['times_rented'] = df_film['times_rented'].astype(int)
df_film.head()

,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,times_rented
film_id,,,,,,,,,,,,,
103,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,1,None,7,4.99,133,27.99,PG,"{Deleted Scenes, Commentaries}",2006-02-15 05:03:42,34
738,ROCKETEER MOTHER,A Awe-Inspiring Character Study of a Robot And...,2006,1,None,3,0.99,178,27.99,PG-13,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42,33
730,RIDGEMONT SUBMARINE,A Unbelieveable Drama of a Waitress And a Comp...,2006,1,None,3,0.99,46,28.99,PG-13,"{Deleted Scenes, Behind the Scenes, Commentaries}",2006-02-15 05:03:42,32
382,GRIT CLOCKWORK,A Thoughtful Display of a Dentist And a Squirr...,2006,1,None,3,0.99,137,21.99,PG,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42,32
489,JUGGLER HARDLY,A Epic Story of a Mad Cow And a Astronaut who ...,2006,1,None,4,0.99,54,14.99,PG-13,"{Trailers, Commentaries}",2006-02-15 05:03:42,32
